In [1]:
import os
print(os.getcwd())

C:\Users\vvenugopalu\Downloads


In [4]:
os.chdir('D:\\tutorials\\DS\\programming')

In [5]:
print(os.getcwd())

D:\tutorials\DS\programming


In [6]:
#### Importing necessary Libraries
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [7]:
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [8]:
print(df.head())
print('***********************************************************************************************************************')
print(df.columns)
print('***********************************************************************************************************************')
print(df.info())
print('***********************************************************************************************************************')
print(df.describe(include='all'))
print('***********************************************************************************************************************')

   id  player_fifa_api_id  player_api_id                 date  overall_rating  \
0   1              218353         505942  2016-02-18 00:00:00            67.0   
1   2              218353         505942  2015-11-19 00:00:00            67.0   
2   3              218353         505942  2015-09-21 00:00:00            62.0   
3   4              218353         505942  2015-03-20 00:00:00            61.0   
4   5              218353         505942  2007-02-22 00:00:00            61.0   

   potential preferred_foot attacking_work_rate defensive_work_rate  crossing  \
0       71.0          right              medium              medium      49.0   
1       71.0          right              medium              medium      49.0   
2       66.0          right              medium              medium      49.0   
3       65.0          right              medium              medium      48.0   
4       65.0          right              medium              medium      48.0   

      ...       vision  pe

In [9]:
print(df.shape)
print(df.isna().sum())
print(df.duplicated().any())

(183978, 42)
id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standin

In [10]:
df = df.drop(['id','player_fifa_api_id','player_api_id','date'],axis=1)
df.dropna(subset=['overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'], how='all', inplace=True)

print(df.isna().sum())
print(df['attacking_work_rate'].value_counts())
print(df['attacking_work_rate'].unique())
df['attacking_work_rate'].fillna(method='ffill',inplace=True,axis=0)
print(df['attacking_work_rate'].value_counts())
print(df['attacking_work_rate'].unique())

overall_rating            0
potential                 0
preferred_foot            0
attacking_work_rate    2394
defensive_work_rate       0
crossing                  0
finishing                 0
heading_accuracy          0
short_passing             0
volleys                1877
dribbling                 0
curve                  1877
free_kick_accuracy        0
long_passing              0
ball_control              0
acceleration              0
sprint_speed              0
agility                1877
reactions                 0
balance                1877
shot_power                0
jumping                1877
stamina                   0
strength                  0
long_shots                0
aggression                0
interceptions             0
positioning               0
vision                 1877
penalties                 0
marking                   0
standing_tackle           0
sliding_tackle         1877
gk_diving                 0
gk_handling               0
gk_kicking          

In [11]:
attacking_work_rate_dummies = pd.get_dummies(df['attacking_work_rate'],prefix='attacking_work_rate').iloc[:, 1:]
attacking_work_rate_dummies.head(2)

,attacking_work_rate_high,attacking_work_rate_le,attacking_work_rate_low,attacking_work_rate_medium,attacking_work_rate_norm,attacking_work_rate_stoc,attacking_work_rate_y
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0


In [12]:
df['volleys'].fillna((df['volleys'].mean()), inplace=True)
df['volleys'].isna().sum()

df['curve'].fillna((df['curve'].mean()), inplace=True)
df['curve'].isna().sum()

df['agility'].fillna((df['agility'].mean()), inplace=True)
df['agility'].isna().sum()

df['balance'].fillna((df['balance'].mean()), inplace=True)
df['balance'].isna().sum()

df['jumping'].fillna((df['jumping'].mean()), inplace=True)
df['jumping'].isna().sum()

df['vision'].fillna((df['vision'].mean()), inplace=True)
df['vision'].isna().sum()

df['sliding_tackle'].fillna((df['sliding_tackle'].mean()), inplace=True)
df['sliding_tackle'].isna().sum()

print(df['overall_rating'].unique())

[67. 62. 61. 74. 73. 71. 70. 69. 68. 65. 64. 54. 51. 52. 47. 53. 66. 59.
 75. 72. 76. 78. 77. 79. 60. 80. 81. 82. 84. 48. 63. 83. 55. 58. 50. 56.
 49. 57. 42. 46. 45. 85. 44. 86. 89. 87. 88. 91. 40. 90. 41. 43. 38. 93.
 92. 39. 33. 36. 37. 35. 94.]


In [13]:
print(df.info())

preferred_foot_dummies = pd.get_dummies(df['preferred_foot'],prefix='preferred_foot')
print(preferred_foot_dummies.head(2))
print(df['defensive_work_rate'].value_counts())

defensive_work_rate_dummies = pd.get_dummies(df['defensive_work_rate'],prefix='defensive_work_rate').iloc[:, 1:]
print(defensive_work_rate_dummies.head(2))

attacking_work_rate_dummies = pd.get_dummies(df['attacking_work_rate'],prefix='attacking_work_rate').iloc[:, 1:]
print(attacking_work_rate_dummies.head(2))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 38 columns):
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    183142 non-null object
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                183142 non-null float64
dribbling              183142 non-null float64
curve                  183142 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
acceleration           183142 non-null float64
sprint_speed           183142 non-null float64
agility                183142 non-null float64
reactions              183142 non-null float64

In [14]:
df = df.drop(['preferred_foot','attacking_work_rate','defensive_work_rate'],axis=1)
print(df.columns)
df = pd.concat([df, attacking_work_rate_dummies,defensive_work_rate_dummies,preferred_foot_dummies], axis=1)
print(df['overall_rating'].unique())
print(df.info())
print(df.isna().sum().any())

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')
[67. 62. 61. 74. 73. 71. 70. 69. 68. 65. 64. 54. 51. 52. 47. 53. 66. 59.
 75. 72. 76. 78. 77. 79. 60. 80. 81. 82. 84. 48. 63. 83. 55. 58. 50. 56.
 49. 57. 42. 46. 45. 85. 44. 86. 89. 87. 88. 91. 40. 90. 41. 43. 38. 93.
 92. 39. 33. 36. 37. 35. 94.]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 62 columns):
overall_rating                183142 non-null float64
potent

In [15]:
print(df.columns)

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'attacking_work_rate_high', 'attacking_work_rate_le',
       'attacking_work_rate_low', 'attacking_work_rate_medium',
       'attacking_work_rate_norm', 'attacking_work_rate_stoc',
       'attacking_work_rate_y', 'defensive_work_rate_1',
       'defensive_work_rate_2', 'defensive_work_rate_3',
       'defensive_work_rate_4', 'defensive_work_rate_5',
       'defensive_work_rate_6', 'defensive_work_rate_7',
       'defensive_work_rate

In [16]:
df = pd.concat([df, attacking_work_rate_dummies,defensive_work_rate_dummies,preferred_foot_dummies], axis=1)
print(df['overall_rating'].unique())
print(df.info())
print(df.isna().sum().any())

[67. 62. 61. 74. 73. 71. 70. 69. 68. 65. 64. 54. 51. 52. 47. 53. 66. 59.
 75. 72. 76. 78. 77. 79. 60. 80. 81. 82. 84. 48. 63. 83. 55. 58. 50. 56.
 49. 57. 42. 46. 45. 85. 44. 86. 89. 87. 88. 91. 40. 90. 41. 43. 38. 93.
 92. 39. 33. 36. 37. 35. 94.]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 89 columns):
overall_rating                183142 non-null float64
potential                     183142 non-null float64
crossing                      183142 non-null float64
finishing                     183142 non-null float64
heading_accuracy              183142 non-null float64
short_passing                 183142 non-null float64
volleys                       183142 non-null float64
dribbling                     183142 non-null float64
curve                         183142 non-null float64
free_kick_accuracy            183142 non-null float64
long_passing                  183142 non-null float64
ball_control                  183142 non-null 

In [18]:
print(df.columns)
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
df_new = ['potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'attacking_work_rate_high', 'attacking_work_rate_le',
       'attacking_work_rate_low', 'attacking_work_rate_medium',
       'attacking_work_rate_norm', 'attacking_work_rate_stoc',
       'attacking_work_rate_y', 'defensive_work_rate_1',
       'defensive_work_rate_2', 'defensive_work_rate_3',
       'defensive_work_rate_4', 'defensive_work_rate_5',
       'defensive_work_rate_6', 'defensive_work_rate_7',
       'defensive_work_rate_8', 'defensive_work_rate_9',
       'defensive_work_rate__0', 'defensive_work_rate_ean',
       'defensive_work_rate_es', 'defensive_work_rate_high',
       'defensive_work_rate_low', 'defensive_work_rate_medium',
       'defensive_work_rate_o', 'defensive_work_rate_ormal',
       'defensive_work_rate_tocky', 'preferred_foot_right','preferred_foot_left']
df_new2 = df[df_new]
new_y = df['overall_rating']

lm.fit(df_new2,new_y)
print(lm.coef_)
print(lm.intercept_)
print(lm.score(df_new2,new_y))

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'attacking_work_rate_high', 'attacking_work_rate_le',
       'attacking_work_rate_low', 'attacking_work_rate_medium',
       'attacking_work_rate_norm', 'attacking_work_rate_stoc',
       'attacking_work_rate_y', 'defensive_work_rate_1',
       'defensive_work_rate_2', 'defensive_work_rate_3',
       'defensive_work_rate_4', 'defensive_work_rate_5',
       'defensive_work_rate_6', 'defensive_work_rate_7',
       'defensive_work_rate

In [19]:
import statsmodels.formula.api as smf
lm = smf.ols(formula='overall_rating ~ potential+crossing+finishing+heading_accuracy+short_passing+volleys\
         +dribbling+curve+free_kick_accuracy+long_passing+ball_control+acceleration+sprint_speed+agility\
         +reactions+balance+shot_power+jumping+stamina+strength+long_shots+aggression+interceptions+positioning\
         +vision+penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+gk_kicking+gk_positioning\
         +gk_reflexes+attacking_work_rate_high+attacking_work_rate_le+attacking_work_rate_low+attacking_work_rate_medium\
         +attacking_work_rate_norm+attacking_work_rate_stoc+attacking_work_rate_y+defensive_work_rate_1\
         +defensive_work_rate_2+defensive_work_rate_3+defensive_work_rate_4+defensive_work_rate_5+defensive_work_rate_6\
         +defensive_work_rate_7+defensive_work_rate_8+defensive_work_rate_9+defensive_work_rate__0\
         +defensive_work_rate_ean+defensive_work_rate_es+defensive_work_rate_high+defensive_work_rate_low\
         +defensive_work_rate_medium+defensive_work_rate_o+defensive_work_rate_ormal+defensive_work_rate_tocky\
         +preferred_foot_right+preferred_foot_left',data=df).fit()
print(lm.conf_int())
print(lm.summary())

                                      0         1
Intercept                     -2.054929 -1.624243
potential                      0.374226  0.379894
crossing                       0.020957  0.024233
finishing                      0.010156  0.013715
heading_accuracy               0.067228  0.070398
short_passing                  0.048897  0.054222
volleys                        0.000658  0.003840
dribbling                     -0.012513 -0.008002
curve                          0.009876  0.012980
free_kick_accuracy             0.012316  0.015065
long_passing                   0.004667  0.008302
ball_control                   0.130334  0.136464
acceleration                   0.002847  0.008207
sprint_speed                   0.008756  0.013844
agility                       -0.010655 -0.006710
reactions                      0.204252  0.208450
balance                        0.005262  0.008313
shot_power                     0.014620  0.018019
jumping                        0.014318  0.017163


In [25]:
y = df['overall_rating']
x = df.drop(['overall_rating'],axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print(x_train.shape, x_test.shape, y_test.shape, y_test.shape)

model2 = LinearRegression()
print(model2.fit(x_train, y_train))

predicted = model2.predict(x_test)
print(predicted)
print('**************************************************************************************************************')
print('Score: ', model2.score(x_test,y_test))

(128199, 88) (54943, 88) (54943,) (54943,)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
[74.60663865 52.17614341 80.92024212 ... 73.89922724 70.71023893
 69.67020058]
**************************************************************************************************************
Score:  0.8429083123335539
